# PIP INSTALL


In [1]:
pip install transformers datasets scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install tensorboard


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install evaluate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!pip install seqeval



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
pip install tf-keras


Note: you may need to restart the kernel to use updated packages.Collecting tf-keras
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------------ --- 1.6/1.7 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 10.5 MB/s eta 0:00:00




[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# GPT 2 Multitask classification


## Import Libraries


In [5]:
import torch
torch.cuda.empty_cache()


In [10]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2Model, GPT2TokenizerFast, Trainer
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import DataCollatorForTokenClassification
from sklearn.metrics import accuracy_score, f1_score, classification_report
import json
import ast  # Import ast for literal evaluation
# Tokenize and preprocess data


In [11]:
dataset_path = "../dataset/processed_dataset.csv"
# Check if CUDA (GPU) is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Load and Preprocess Data


In [12]:


# Load dataset from CSV
data = pd.read_csv(dataset_path)

# Convert ner_labels from string to list of dictionaries
data["ner_labels"] = data["ner_labels"].apply(lambda x: ast.literal_eval(x) if x else [])

# Initialize tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Encode intent, category, and NER labels
intent_encoder = LabelEncoder()
category_encoder = LabelEncoder()
ner_encoder = LabelEncoder()

# Fit encoders
all_intent_labels = data["intent"].tolist()
all_category_labels = data["category"].tolist()
all_ner_labels = ["O"] + [label for sublist in data["ner_labels"] for label in [item["label"] for item in sublist]]

# Add "I-" prefixes for subsequent subword tokens before fitting
all_ner_labels_with_prefixes = all_ner_labels + [f"I-{label}" for label in all_ner_labels if label != "O"]

# Fit the encoders
intent_encoder.fit(all_intent_labels)
category_encoder.fit(all_category_labels)
ner_encoder.fit(all_ner_labels_with_prefixes)  # Fit on all possible labels, including "I-" prefixes

# Test the encoder (should now work without errors)
print(ner_encoder.transform(["O"]))  # Should return [0]
print(ner_encoder.transform(["order_number"]))  # Should return a valid ID
print(ner_encoder.transform(["I-order_number"]))  # Should return a valid ID

def preprocess_ner_annotations(text, ner_annotations):
    # Tokenize the text with GPT-2 tokenizer
    encoding = tokenizer(
        text,
        padding=False,
        truncation=False,
        return_offsets_mapping=True,  # Include offsets to align tokens with original text
        return_tensors=None
    )
    tokens = encoding["input_ids"]
    offset_mapping = encoding["offset_mapping"]

    # Initialize NER labels for each token as "O" (outside entity)
    token_ner_labels = ["O"] * len(tokens)

    # Assign NER labels to the corresponding tokens
    remaining_text = text
    for annotation in ner_annotations:
        ner_text = annotation["text"]
        ner_label = annotation["label"]

        # Find the position of the annotated text in the remaining text
        start_idx = remaining_text.find(ner_text)
        if start_idx == -1:
            continue  # Skip if the annotation is not found

        end_idx = start_idx + len(ner_text)

        # Update the token-level NER labels
        for i, (start_offset, end_offset) in enumerate(offset_mapping):
            if start_offset >= start_idx and end_offset <= end_idx:
                if start_offset == start_idx:
                    token_ner_labels[i] = ner_label  # Assign label to the first token
                else:
                    token_ner_labels[i] = "I-" + ner_label  # Use "I-" prefix for subsequent tokens

        # Update the remaining text
        remaining_text = remaining_text[end_idx:]

    return tokens, token_ner_labels

# Apply the preprocessing function to the DataFrame
data["tokens"], data["token_ner_labels"] = zip(*data.apply(lambda row: preprocess_ner_annotations(row["instruction"], row["ner_labels"]), axis=1))

# Convert the token_ner_labels to numerical IDs
data["token_ner_ids"] = data["token_ner_labels"].apply(lambda labels: [ner_encoder.transform([label])[0] for label in labels])

# Convert the tokens to tensors
data["tokens"] = data["tokens"].apply(lambda x: torch.tensor(x, dtype=torch.long))

# Convert the token_ner_ids to tensors
data["token_ner_ids"] = data["token_ner_ids"].apply(lambda x: torch.tensor(x, dtype=torch.long))

# Convert the intent and category labels to numerical IDs
data["intent_id"] = data["intent"].apply(lambda x: intent_encoder.transform([x])[0])
data["category_id"] = data["category"].apply(lambda x: category_encoder.transform([x])[0])

# Convert the intent_id and category_id to tensors
data["intent_id"] = data["intent_id"].apply(lambda x: torch.tensor(x, dtype=torch.long))
data["category_id"] = data["category_id"].apply(lambda x: torch.tensor(x, dtype=torch.long))

# Print the first few rows of the DataFrame to verify
print(data.head())

data.to_csv("../dataset/processed_dataset_with_tensors.csv", index=False)


[9]
[16]
[6]
                                         instruction category        intent  \
0          question about cancelling order ORD-71383    ORDER  cancel_order   
1  i have a question about cancelling oorder ORD-...    ORDER  cancel_order   
2           i need help cancelling puchase ORD-48300    ORDER  cancel_order   
3                i need to cancel purchase ORD-65724    ORDER  cancel_order   
4  i cannot afford this order, cancel purchase OR...    ORDER  cancel_order   

                                            response  has_placeholder  \
0  I've understood you have a question regarding ...             True   
1  I've been informed that you have a question ab...             True   
2  I can sense that you're seeking assistance wit...             True   
3  I understood that you need assistance with can...             True   
4  I'm sensitive to the fact that you're facing f...             True   

                                         ner_labels  \
0  [{'text': 'ORD-

In [ ]:
def preprocess_example(row):
    text = row["text"]
    intent_label = intent_encoder.transform([row["intent"]])[0]
    category_label = category_encoder.transform([row["category"]])[0]
    ner_annotations = row["ner_labels"]

    # Preprocess NER annotations at the token level
    tokens, token_ner_labels = preprocess_ner_annotations(text, ner_annotations)

    # Tokenize the text with padding and truncation
    encoding = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_offsets_mapping=False,  # Offset mapping is not needed here
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].squeeze(0)
    attention_mask = encoding['attention_mask'].squeeze(0)

    # Align token-level NER labels with padded/truncated tokens
    aligned_ner_labels = []
    for i, token_id in enumerate(input_ids):
        if i < len(tokens):
            label = token_ner_labels[i]
            aligned_ner_labels.append(ner_encoder.transform([label])[0])
        else:
            aligned_ner_labels.append(-100)  # Padding tokens are ignored

    # Convert aligned NER labels to a tensor
    aligned_ner_labels = torch.tensor(aligned_ner_labels, dtype=torch.long)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "intent_label": intent_label,
        "category_label": category_label,
        "ner_labels": aligned_ner_labels
    }


In [ ]:
# def preprocess_data(data):
#     processed_data = []
#     for _, row in data.iterrows():
#         text = row["text"]
#         intent_label = intent_encoder.transform([row["intent"]])[0]
#         category_label = category_encoder.transform([row["category"]])[0]
#         ner_annotations = row["ner_labels"]

#         # Preprocess NER annotations at the token level
#         tokens, token_ner_labels = preprocess_ner_annotations(text, ner_annotations)

#         # Tokenize the text with padding and truncation
#         encoding = tokenizer(
#             text,
#             padding='max_length',
#             truncation=True,
#             max_length=128,
#             return_offsets_mapping=False,  # Offset mapping is not needed here
#             return_tensors='pt'
#         )
#         input_ids = encoding['input_ids'].squeeze(0)
#         attention_mask = encoding['attention_mask'].squeeze(0)

#         # Align token-level NER labels with padded/truncated tokens
#         aligned_ner_labels = []
#         for i, token_id in enumerate(input_ids):
#             if i < len(tokens):
#                 label = token_ner_labels[i]
#                 aligned_ner_labels.append(ner_encoder.transform([label])[0])
#             else:
#                 aligned_ner_labels.append(-100)  # Padding tokens are ignored

#         # Convert aligned NER labels to a tensor
#         aligned_ner_labels = torch.tensor(aligned_ner_labels, dtype=torch.long)

#         # # Print the results for debugging
#         # print("Input IDs:", input_ids)
#         # print("Tokens:", tokenizer.convert_ids_to_tokens(input_ids))
#         # print("NER Labels:", aligned_ner_labels)
#         # print("-" * 80)

#         # Append the processed example to the dataset
#         processed_data.append({
#             "input_ids": input_ids,
#             "attention_mask": attention_mask,
#             "intent_label": intent_label,
#             "category_label": category_label,
#             "ner_labels": aligned_ner_labels
#         })

#     return processed_data


# # # Process the dataset
# # train_dataset = preprocess_data(data)


## Define DataSet


In [ ]:
from torch.utils.data import Dataset, DataLoader

class MultiTaskDataset(Dataset):
    def __init__(self, data):
        self.data = data  # Store the raw data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        processed_example = preprocess_example(row)
        return {
            "input_ids": torch.tensor(processed_example["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(processed_example["attention_mask"], dtype=torch.long),
            "intent_label": torch.tensor(processed_example["intent_label"], dtype=torch.long),
            "category_label": torch.tensor(processed_example["category_label"], dtype=torch.long),
            "ner_labels": torch.tensor(processed_example["ner_labels"], dtype=torch.long)
        }


## Splitting Dataset


In [ ]:
# Split the data before preprocessing
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Create datasets
train_dataset = MultiTaskDataset(train_data)
val_dataset = MultiTaskDataset(val_data)
test_dataset = MultiTaskDataset(test_data)


In [ ]:
# Flatten all NER labels and check their unique values
all_ner_labels = [label for example in train_dataset for label in example["ner_labels"].tolist() if label != -100]
unique_ner_labels = set(all_ner_labels)

print("Unique NER Labels:", unique_ner_labels)


Streaming output truncated to the last 5000 lines.
<ipython-input-63-25156cdb9210>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "ner_labels": torch.tensor(processed_example["ner_labels"], dtype=torch.long)
<ipython-input-63-25156cdb9210>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(processed_example["input_ids"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(processed_example["attention_mask"], dtype=torch.long),
<ipyt

Unique NER Labels: {0, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 18}


<ipython-input-63-25156cdb9210>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(processed_example["input_ids"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(processed_example["attention_mask"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "ner_labels": torch.tensor(processed_example["ner_labels"], dtype=torch.long)
<ipython-input-63-25156cdb9210>:14: UserWarning: To copy

In [ ]:
# Define the number of classes for each task
num_intents = len(intent_encoder.classes_)
num_categories = len(category_encoder.classes_)
num_ner_labels = len(ner_encoder.classes_)


## Model


In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2Model

class MultiTaskGPT2(GPT2Model):
    def __init__(self, config, num_intents, num_categories, num_ner_labels):
        super().__init__(config)
        self.intent_classifier = nn.Linear(config.hidden_size, num_intents)
        self.category_classifier = nn.Linear(config.hidden_size, num_categories)
        self.ner_classifier = nn.Linear(config.hidden_size, num_ner_labels)

    def forward(self, input_ids, attention_mask=None, intent_label=None, category_label=None, ner_labels=None,
                intent_class_weights=None, category_class_weights=None, ner_class_weights=None):
        # Forward pass through GPT-2
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)

        # Use the last token's hidden state for intent and category classification
        cls_hidden_state = hidden_states[:, -1, :]  # Shape: (batch_size, hidden_size)
        intent_logits = self.intent_classifier(cls_hidden_state)  # Shape: (batch_size, num_intents)
        category_logits = self.category_classifier(cls_hidden_state)  # Shape: (batch_size, num_categories)

        # Use all token hidden states for NER classification
        ner_logits = self.ner_classifier(hidden_states)  # Shape: (batch_size, seq_len, num_ner_labels)

        # Compute loss if labels are provided
        loss = None
        if intent_label is not None and category_label is not None and ner_labels is not None:
            intent_loss_fn = nn.CrossEntropyLoss(weight=intent_class_weights)
            category_loss_fn = nn.CrossEntropyLoss(weight=category_class_weights)
            ner_loss_fn = nn.CrossEntropyLoss(weight=ner_class_weights, ignore_index=-100)

            intent_loss = intent_loss_fn(intent_logits, intent_label)
            category_loss = category_loss_fn(category_logits, category_label)
            ner_loss = ner_loss_fn(ner_logits.view(-1, ner_logits.shape[-1]), ner_labels.view(-1))

            loss = intent_loss + category_loss + ner_loss

        return (loss, intent_logits, category_logits, ner_logits)


## Initialize Model


In [ ]:
# Define the number of classes for each task
num_intents = len(intent_encoder.classes_)
num_categories = len(category_encoder.classes_)
num_ner_labels = len(ner_encoder.classes_)

# Initialize the model
model = MultiTaskGPT2.from_pretrained('gpt2', num_intents=num_intents, num_categories=num_categories, num_ner_labels=num_ner_labels)
model.resize_token_embeddings(len(tokenizer))  # Resize embeddings if you added a new padding token
model.to(device)  # Move the model to GPU


Some weights of MultiTaskGPT2 were not initialized from the model checkpoint at gpt2 and are newly initialized: ['category_classifier.bias', 'category_classifier.weight', 'intent_classifier.bias', 'intent_classifier.weight', 'ner_classifier.bias', 'ner_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MultiTaskGPT2(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (intent_classifier): Linear(in_features=768, out_features=27, bias=True)
  (category_classifier): Linear(in_features=768, out_features=11, bias=True)
  (ner_classifier): Linear(in_features=768, ou

## Class Weight


In [ ]:
import numpy as np
import torch

# Intent class weights
intent_class_counts = np.bincount([example["intent_label"] for example in train_dataset])
intent_class_weights = 1.0 / intent_class_counts
intent_class_weights = intent_class_weights / np.sum(intent_class_weights)
intent_class_weights = torch.tensor(intent_class_weights, dtype=torch.float).to(device)

# Category class weights
category_class_counts = np.bincount([example["category_label"] for example in train_dataset])
category_class_weights = 1.0 / category_class_counts
category_class_weights = category_class_weights / np.sum(category_class_weights)
category_class_weights = torch.tensor(category_class_weights, dtype=torch.float).to(device)

# NER class weights
ner_labels_flattened = [label for example in train_dataset for label in example["ner_labels"].tolist() if label != -100]
ner_class_counts = np.bincount(ner_labels_flattened)
ner_class_weights = 1.0 / ner_class_counts
ner_class_weights = ner_class_weights / np.sum(ner_class_weights)
ner_class_weights = torch.tensor(ner_class_weights, dtype=torch.float).to(device)

# Print the shapes of the class weights to verify correctness
print("Intent Class Weights Shape:", intent_class_weights.shape)
print("Category Class Weights Shape:", category_class_weights.shape)
print("NER Class Weights Shape:", ner_class_weights.shape)


Streaming output truncated to the last 5000 lines.
<ipython-input-63-25156cdb9210>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "ner_labels": torch.tensor(processed_example["ner_labels"], dtype=torch.long)
<ipython-input-63-25156cdb9210>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(processed_example["input_ids"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(processed_example["attention_mask"], dtype=torch.long),
<ipyt

Intent Class Weights Shape: torch.Size([27])
Category Class Weights Shape: torch.Size([11])
NER Class Weights Shape: torch.Size([19])


<ipython-input-63-25156cdb9210>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(processed_example["input_ids"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(processed_example["attention_mask"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "ner_labels": torch.tensor(processed_example["ner_labels"], dtype=torch.long)
<ipython-input-63-25156cdb9210>:14: UserWarning: To copy

## Training Arguments


In [ ]:
from transformers import TrainingArguments

# Define optimized training arguments
training_args = TrainingArguments(
    output_dir="./results_v1",                  # Directory to save model checkpoints
    num_train_epochs=4,                         # Number of training epochs
    per_device_train_batch_size=16,              # Reduce batch size to prevent OOM
    per_device_eval_batch_size=16,               # Keep evaluation batch size low
    gradient_accumulation_steps=2,              # Accumulate gradients to simulate larger batch size
    gradient_checkpointing=True,                # Enable gradient checkpointing to save memory
    learning_rate=3e-5,                         # Slightly reduced learning rate for stability
    weight_decay=0.01,                          # Weight decay for regularization
    warmup_steps=500,                           # Warm-up to stabilize initial training
    logging_dir="./logs",                       # Directory for storing logs
    logging_steps=50,                           # Log every 50 steps to reduce overhead
    evaluation_strategy="epoch",                # Evaluate at the end of each epoch
    save_strategy="epoch",                      # Save checkpoint at the end of each epoch
    save_total_limit=2,                         # Keep only the last 2 checkpoints
    load_best_model_at_end=True,                # Load the best model at the end of training
    metric_for_best_model="eval_loss",          # Use evaluation loss to determine the best model
    greater_is_better=False,                    # Lower eval_loss is better
    fp16=True,                                  # Use mixed precision training (if GPU supports it)
    fp16_full_eval=True,                        # Use fp16 for evaluation to save memory
    report_to="tensorboard",                    # Report metrics to TensorBoard
    save_safetensors=True,                      # Save models in `safetensors` format for efficiency
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Metrics Computation


In [ ]:
import evaluate

# Load metrics using the evaluate library
accuracy_metric = evaluate.load("accuracy")
seqeval_metric = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    seqeval = seqeval_metric.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy["accuracy"], "seqeval": seqeval["f1"]}

# Load metrics using the evaluate library
accuracy_metric = evaluate.load("accuracy")
seqeval_metric = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    intent_logits, category_logits, ner_logits = logits
    intent_labels, category_labels, ner_labels = labels

    # Compute intent accuracy
    intent_preds = np.argmax(intent_logits, axis=-1)
    intent_results = accuracy_metric.compute(predictions=intent_preds, references=intent_labels)

    # Compute category accuracy
    category_preds = np.argmax(category_logits, axis=-1)
    category_results = accuracy_metric.compute(predictions=category_preds, references=category_labels)

    # Compute NER metrics
    ner_preds = np.argmax(ner_logits, axis=-1)
    true_ner_labels = [[ner_encoder.inverse_transform([label]) for label in seq if label != -100] for seq in ner_labels]
    pred_ner_labels = [[ner_encoder.inverse_transform([pred]) for pred, label in zip(seq_preds, seq_labels) if label != -100] for seq_preds, seq_labels in zip(ner_preds, ner_labels)]
    ner_results = seqeval_metric.compute(predictions=pred_ner_labels, references=true_ner_labels)

    # Combine results
    return {
        "intent_accuracy": intent_results["accuracy"],
        "category_accuracy": category_results["accuracy"],
        "ner_precision": ner_results["overall_precision"],
        "ner_recall": ner_results["overall_recall"],
        "ner_f1": ner_results["overall_f1"]
    }


## Data Collator


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding='max_length',  # Explicitly set padding to max_length
    max_length=64,
    pad_to_multiple_of=8
)


## Pass Class wiegh Trainer


In [ ]:

class WeightedLossTrainer(Trainer):
    def __init__(self, intent_class_weights, category_class_weights, ner_class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.intent_class_weights = intent_class_weights
        self.category_class_weights = category_class_weights
        self.ner_class_weights = ner_class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None): # Add num_items_in_batch as an argument
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        intent_labels = inputs['intent_label']
        category_labels = inputs['category_label']
        ner_labels = inputs['ner_labels']

        # Forward pass through the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            intent_label=intent_labels,
            category_label=category_labels,
            ner_labels=ner_labels,
            intent_class_weights=self.intent_class_weights,
            category_class_weights=self.category_class_weights,
            ner_class_weights=self.ner_class_weights
        )

        loss = outputs[0]  # Extract the loss
        if return_outputs:
            return (loss, outputs[1:])  # Return loss and other outputs
        else:
            return loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None, num_items_in_batch=None):  # Add ignore_keys
        # Override prediction_step to return all outputs for compute_metrics
        loss, (intent_logits, category_logits, ner_logits) = self.compute_loss(model, inputs, return_outputs=True)

        # Following the Trainer base class to collect outputs during evaluation
        return (loss, (intent_logits, category_logits, ner_logits), inputs["ner_labels"])


## Train the Model


In [ ]:
trainer = WeightedLossTrainer(
    intent_class_weights=intent_class_weights,
    category_class_weights=category_class_weights,
    ner_class_weights=ner_class_weights,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
# for batch in train_loader:
#     input_ids = batch["input_ids"].to(device)
#     attention_mask = batch["attention_mask"].to(device)
#     intent_label = batch["intent_label"].to(device)
#     category_label = batch["category_label"].to(device)
#     ner_labels = batch["ner_labels"].to(device)

#     outputs = model(input_ids, attention_mask, intent_label, category_label, ner_labels)

trainer.train()
eval_results = trainer.evaluate()
print(eval_results)


<ipython-input-63-25156cdb9210>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(processed_example["input_ids"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(processed_example["attention_mask"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "ner_labels": torch.tensor(processed_example["ner_labels"], dtype=torch.long)
<ipython-input-63-25156cdb9210>:14: UserWarning: To copy

Epoch,Training Loss,Validation Loss


Streaming output truncated to the last 5000 lines.
<ipython-input-63-25156cdb9210>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "ner_labels": torch.tensor(processed_example["ner_labels"], dtype=torch.long)
<ipython-input-63-25156cdb9210>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(processed_example["input_ids"], dtype=torch.long),
<ipython-input-63-25156cdb9210>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(processed_example["attention_mask"], dtype=torch.long),
<ipyt

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 8.12 MiB is free. Process 55088 has 14.73 GiB memory in use. Of the allocated memory 14.44 GiB is allocated by PyTorch, and 157.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!nvidia-smi


## Evaluate


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")


# Full Code


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import GPT2TokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import ast
from transformers import DataCollatorForTokenClassification
import evaluate
import gc  # For explicit garbage collection
import torch.nn as nn
from transformers import GPT2Model

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LazyMultiTaskDataset(Dataset):
    def __init__(self, data, tokenizer, intent_encoder, category_encoder, ner_encoder, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.intent_encoder = intent_encoder
        self.category_encoder = category_encoder
        self.ner_encoder = ner_encoder
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def preprocess_ner_annotations(self, text, ner_annotations):
        encoding = self.tokenizer(
            text,
            padding=False,
            truncation=False,
            return_offsets_mapping=True,
            return_tensors=None
        )
        tokens = encoding["input_ids"]
        offset_mapping = encoding["offset_mapping"]
        token_ner_labels = ["O"] * len(tokens)

        remaining_text = text
        for annotation in ner_annotations:
            ner_text = annotation["text"]
            ner_label = annotation["label"]

            start_idx = remaining_text.find(ner_text)
            if start_idx == -1:
                continue

            end_idx = start_idx + len(ner_text)

            for i, (start_offset, end_offset) in enumerate(offset_mapping):
                if start_offset >= start_idx and end_offset <= end_idx:
                    token_ner_labels[i] = ner_label if start_offset == start_idx else "I-" + ner_label

            remaining_text = remaining_text[end_idx:]

        return tokens, token_ner_labels

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row["text"]
        intent_label = self.intent_encoder.transform([row["intent"]])[0]
        category_label = self.category_encoder.transform([row["category"]])[0]
        ner_annotations = ast.literal_eval(row["ner_labels"])

        # Process NER annotations
        tokens, token_ner_labels = self.preprocess_ner_annotations(text, ner_annotations)

        # Tokenize with padding
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Align NER labels
        aligned_ner_labels = []
        for i in range(self.max_length):
            if i < len(tokens):
                label = token_ner_labels[i]
                aligned_ner_labels.append(self.ner_encoder.transform([label])[0])
            else:
                aligned_ner_labels.append(-100)

        return {
            "input_ids": encoding['input_ids'].squeeze(0),
            "attention_mask": encoding['attention_mask'].squeeze(0),
            "intent_label": torch.tensor(intent_label, dtype=torch.long),
            "category_label": torch.tensor(category_label, dtype=torch.long),
            "ner_labels": torch.tensor(aligned_ner_labels, dtype=torch.long)
        }


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,5.186100,4.307321
2,0.212700,0.055225
3,0.066600,0.023593
4,0.027500,0.018798


{'eval_loss': 0.01879827491939068, 'eval_runtime': 6.7036, 'eval_samples_per_second': 306.551, 'eval_steps_per_second': 19.243, 'epoch': 4.0}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import shutil

# Define source and destination paths
source = "./final_model"
destination = "/content/drive/My Drive/final_model"

# Move the folder
shutil.move(source, destination)

print("Folder moved successfully!")


Folder moved successfully!


In [ ]:
import os

path = "/content/drive/My Drive/final_model"
if os.path.exists(path):
    print("Folder found at:", path)
else:
    print("Folder not found.")



Folder found at: /content/drive/My Drive/final_model


In [ ]:
import shutil

# Define the source folder path within Google Drive
source_folder = '/content/gpt_v2'

# Compress the folder into a zip file
shutil.make_archive('gpt_v2', 'zip', source_folder) # Changed source folder

# Download the zip file
from google.colab import files
files.download('gpt_v2.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from transformers import GPT2TokenizerFast
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Define custom evaluation metrics
def compute_metrics(predictions, references):
    intent_preds, category_preds, ner_preds = predictions
    intent_labels, category_labels, ner_labels = references

    # Compute Accuracy for Intent and Category
    intent_accuracy = accuracy_score(intent_labels, intent_preds)
    category_accuracy = accuracy_score(category_labels, category_preds)

    # Compute F1 scores for Intent and Category
    intent_f1 = f1_score(intent_labels, intent_preds, average='weighted')
    category_f1 = f1_score(category_labels, category_preds, average='weighted')

    # Compute F1 score for NER (using flat labels for simplicity)
    ner_f1 = f1_score(ner_labels.flatten(), ner_preds.flatten(), average='weighted', labels=np.unique(ner_labels))

    return {
        'intent_accuracy': intent_accuracy,
        'category_accuracy': category_accuracy,
        'intent_f1': intent_f1,
        'category_f1': category_f1,
        'ner_f1': ner_f1
    }

# Load the saved model and tokenizer
model_path = path
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# Define the MultiTaskGPT2 class (same as in your code)
class MultiTaskGPT2(torch.nn.Module):
    def __init__(self, num_intents, num_categories, num_ner_labels):
        super().__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        config = self.gpt2.config
        self.intent_classifier = torch.nn.Linear(config.hidden_size, num_intents)
        self.category_classifier = torch.nn.Linear(config.hidden_size, num_categories)
        self.ner_classifier = torch.nn.Linear(config.hidden_size, num_ner_labels)

        self.gpt2.gradient_checkpointing_enable()

    def forward(self, input_ids, attention_mask=None, intent_label=None, category_label=None, ner_labels=None):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        cls_hidden_state = hidden_states[:, -1, :]

        intent_logits = self.intent_classifier(cls_hidden_state)
        category_logits = self.category_classifier(cls_hidden_state)
        ner_logits = self.ner_classifier(hidden_states)

        loss = None
        if all(label is not None for label in [intent_label, category_label, ner_labels]):
            intent_loss_fn = torch.nn.CrossEntropyLoss()
            category_loss_fn = torch.nn.CrossEntropyLoss()
            ner_loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)

            intent_loss = intent_loss_fn(intent_logits, intent_label)
            category_loss = category_loss_fn(category_logits, category_label)
            ner_loss = ner_loss_fn(ner_logits.view(-1, ner_logits.shape[-1]), ner_labels.view(-1))

            loss = intent_loss + category_loss + ner_loss

        return (loss, intent_logits, category_logits, ner_logits)

# Assuming you have encoded labels and preprocessed data
# Replace these with your LabelEncoder and dataset
intent_encoder = LabelEncoder()
category_encoder = LabelEncoder()
ner_encoder = LabelEncoder()

# Load your validation dataset
# Assuming 'val_data' is your validation DataFrame
val_data = pd.read_csv("val_data.csv")

# Define the dataset and dataloader
class LazyMultiTaskDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, intent_encoder, category_encoder, ner_encoder, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.intent_encoder = intent_encoder
        self.category_encoder = category_encoder
        self.ner_encoder = ner_encoder
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def preprocess_ner_annotations(self, text, ner_annotations):
        encoding = self.tokenizer(
            text,
            padding=False,
            truncation=False,
            return_offsets_mapping=True,
            return_tensors=None
        )
        tokens = encoding["input_ids"]
        offset_mapping = encoding["offset_mapping"]
        token_ner_labels = ["O"] * len(tokens)

        remaining_text = text
        for annotation in ner_annotations:
            ner_text = annotation["text"]
            ner_label = annotation["label"]
            start_idx = remaining_text.find(ner_text)
            if start_idx == -1:
                continue
            end_idx = start_idx + len(ner_text)
            for i, (start_offset, end_offset) in enumerate(offset_mapping):
                if start_offset >= start_idx and end_offset <= end_idx:
                    token_ner_labels[i] = ner_label if start_offset == start_idx else "I-" + ner_label
            remaining_text = remaining_text[end_idx:]

        return tokens, token_ner_labels

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row["text"]
        intent_label = self.intent_encoder.transform([row["intent"]])[0]
        category_label = self.category_encoder.transform([row["category"]])[0]
        ner_annotations = eval(row["ner_labels"])

        # Process NER annotations
        tokens, token_ner_labels = self.preprocess_ner_annotations(text, ner_annotations)

        # Tokenize with padding
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Align NER labels
        aligned_ner_labels = []
        for i in range(self.max_length):
            if i < len(tokens):
                label = token_ner_labels[i]
                aligned_ner_labels.append(self.ner_encoder.transform([label])[0])
            else:
                aligned_ner_labels.append(-100)

        return {
            "input_ids": encoding['input_ids'].squeeze(0),
            "attention_mask": encoding['attention_mask'].squeeze(0),
            "intent_label": torch.tensor(intent_label, dtype=torch.long),
            "category_label": torch.tensor(category_label, dtype=torch.long),
            "ner_labels": torch.tensor(aligned_ner_labels, dtype=torch.long)
        }

# Prepare validation dataset
val_dataset = LazyMultiTaskDataset(val_data, tokenizer, intent_encoder, category_encoder, ner_encoder)
val_dataloader = DataLoader(val_dataset, batch_size=16)

model_path = "/content/gpt_v2/checkpoint-12000"  # Replace with the actual path
model_file = f"{model_path}/model.safetensors"

# Load the trained model
model = MultiTaskGPT2(num_intents=len(intent_encoder.classes_),
                      num_categories=len(category_encoder.classes_),
                      num_ner_labels=len(ner_encoder.classes_))
model.load_state_dict(torch.load(model_file))
model.to(device)

# Set model to evaluation mode
model.eval()

# Initialize lists to store predictions and labels
all_preds_intent = []
all_preds_category = []
all_preds_ner = []
all_labels_intent = []
all_labels_category = []
all_labels_ner = []

# Evaluate the model
with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        intent_labels = batch['intent_label'].to(device)
        category_labels = batch['category_label'].to(device)
        ner_labels = batch['ner_labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        intent_logits, category_logits, ner_logits = outputs[1], outputs[2], outputs[3]

        # Collect predictions
        intent_preds = torch.argmax(intent_logits, dim=1).cpu().numpy()
        category_preds = torch.argmax(category_logits, dim=1).cpu().numpy()
        ner_preds = torch.argmax(ner_logits, dim=2).cpu().numpy()

        # Collect true labels
        all_preds_intent.extend(intent_preds)
        all_preds_category.extend(category_preds)
        all_preds_ner.extend(ner_preds)
        all_labels_intent.extend(intent_labels.cpu().numpy())
        all_labels_category.extend(category_labels.cpu().numpy())
        all_labels_ner.extend(ner_labels.cpu().numpy())

# Compute metrics
metrics = compute_metrics((all_preds_intent, all_preds_category, all_preds_ner),
                          (all_labels_intent, all_labels_category, all_labels_ner))

# Print results
print("Evaluation Metrics:")
for metric, score in metrics.items():
    print(f"{metric}: {score:.4f}")


AttributeError: 'LabelEncoder' object has no attribute 'classes_'

In [ ]:
## Version 2import pandas as pd
import torch
from transformers import GPT2TokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import ast
import shutil
from transformers import DataCollatorForTokenClassification
import evaluate
import gc  # For explicit garbage collection
import torch.nn as nn
from transformers import GPT2Model

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LazyMultiTaskDataset(Dataset):
    def __init__(self, data, tokenizer, intent_encoder, category_encoder, ner_encoder, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.intent_encoder = intent_encoder
        self.category_encoder = category_encoder
        self.ner_encoder = ner_encoder
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def preprocess_ner_annotations(self, text, ner_annotations):
        encoding = self.tokenizer(
            text,
            padding=False,
            truncation=False,
            return_offsets_mapping=True,
            return_tensors=None
        )
        tokens = encoding["input_ids"]
        offset_mapping = encoding["offset_mapping"]
        token_ner_labels = ["O"] * len(tokens)

        remaining_text = text
        for annotation in ner_annotations:
            ner_text = annotation["text"]
            ner_label = annotation["label"]

            start_idx = remaining_text.find(ner_text)
            if start_idx == -1:
                continue

            end_idx = start_idx + len(ner_text)

            for i, (start_offset, end_offset) in enumerate(offset_mapping):
                if start_offset >= start_idx and end_offset <= end_idx:
                    token_ner_labels[i] = ner_label if start_offset == start_idx else "I-" + ner_label

            remaining_text = remaining_text[end_idx:]

        return tokens, token_ner_labels

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row["text"]
        intent_label = self.intent_encoder.transform([row["intent"]])[0]
        category_label = self.category_encoder.transform([row["category"]])[0]
        ner_annotations = ast.literal_eval(row["ner_labels"])

        # Process NER annotations
        tokens, token_ner_labels = self.preprocess_ner_annotations(text, ner_annotations)

        # Tokenize with padding
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Align NER labels
        aligned_ner_labels = []
        for i in range(self.max_length):
            if i < len(tokens):
                label = token_ner_labels[i]
                aligned_ner_labels.append(self.ner_encoder.transform([label])[0])
            else:
                aligned_ner_labels.append(-100)

        return {
            "input_ids": encoding['input_ids'].squeeze(0),
            "attention_mask": encoding['attention_mask'].squeeze(0),
            "intent_label": torch.tensor(intent_label, dtype=torch.long),
            "category_label": torch.tensor(category_label, dtype=torch.long),
            "ner_labels": torch.tensor(aligned_ner_labels, dtype=torch.long)
        }

class MultiTaskGPT2(nn.Module):
    def __init__(self, num_intents, num_categories, num_ner_labels):
        super().__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        config = self.gpt2.config
        self.intent_classifier = nn.Linear(config.hidden_size, num_intents)
        self.category_classifier = nn.Linear(config.hidden_size, num_categories)
        self.ner_classifier = nn.Linear(config.hidden_size, num_ner_labels)

        # Enable gradient checkpointing
        self.gpt2.gradient_checkpointing_enable()

    def forward(self, input_ids, attention_mask=None, intent_label=None, category_label=None, ner_labels=None,
                intent_class_weights=None, category_class_weights=None, ner_class_weights=None):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

        # Free memory
        del outputs
        torch.cuda.empty_cache()

        cls_hidden_state = hidden_states[:, -1, :]
        intent_logits = self.intent_classifier(cls_hidden_state)
        category_logits = self.category_classifier(cls_hidden_state)
        ner_logits = self.ner_classifier(hidden_states)

        # Free memory
        del hidden_states
        torch.cuda.empty_cache()

        loss = None
        if all(label is not None for label in [intent_label, category_label, ner_labels]):
            intent_loss_fn = nn.CrossEntropyLoss(weight=intent_class_weights)
            category_loss_fn = nn.CrossEntropyLoss(weight=category_class_weights)
            ner_loss_fn = nn.CrossEntropyLoss(weight=ner_class_weights, ignore_index=-100)

            intent_loss = intent_loss_fn(intent_logits, intent_label)
            category_loss = category_loss_fn(category_logits, category_label)
            ner_loss = ner_loss_fn(ner_logits.view(-1, ner_logits.shape[-1]), ner_labels.view(-1))

            loss = intent_loss + category_loss + ner_loss

        return (loss, intent_logits, category_logits, ner_logits)

def evaluate_model():
    # Load and prepare data
    dataset_path = "checkpoint_5.csv"
    data = pd.read_csv(dataset_path)

    # Initialize tokenizer
    tokenizer = GPT2TokenizerFast.from_pretrained('/content/final_modelv2')
    tokenizer.pad_token = tokenizer.eos_token

    # Initialize encoders
    intent_encoder = LabelEncoder()
    category_encoder = LabelEncoder()
    ner_encoder = LabelEncoder()

    # Fit encoders
    all_intent_labels = data["intent"].unique()
    all_category_labels = data["category"].unique()

    # Extract unique NER labels
    all_ner_labels = ["O"]
    for ner_list in data["ner_labels"]:
        for item in ast.literal_eval(ner_list):
            if item["label"] not in all_ner_labels:
                all_ner_labels.append(item["label"])

    # Add "I-" prefixes
    all_ner_labels_with_prefixes = all_ner_labels + [f"I-{label}" for label in all_ner_labels if label != "O"]

    # Fit encoders
    intent_encoder.fit(all_intent_labels)
    category_encoder.fit(all_category_labels)
    ner_encoder.fit(all_ner_labels_with_prefixes)

    # Split data
    train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
    val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

    # Create test dataset
    test_dataset = LazyMultiTaskDataset(test_data, tokenizer, intent_encoder, category_encoder, ner_encoder)

    # Initialize model
    model = MultiTaskGPT2(
        num_intents=len(intent_encoder.classes_),
        num_categories=len(category_encoder.classes_),
        num_ner_labels=len(ner_encoder.classes_)
    )
    model.load_state_dict(torch.load('/content/final_modelv2/training_args.bin'))
    model.to(device)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results_v2",
        num_train_epochs=4,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        gradient_accumulation_steps=2,
        learning_rate=3e-5,
        weight_decay=0.01,
        warmup_steps=500,
        logging_dir="./logs",
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True,
        gradient_checkpointing=False,
        dataloader_num_workers=2,
        dataloader_pin_memory=True
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=test_dataset,  # Use test dataset for evaluation
        eval_dataset=test_dataset,
        data_collator=DataCollatorForTokenClassification(
            tokenizer=tokenizer,
            padding='max_length',
            max_length=128,
            pad_to_multiple_of=8
        )
    )

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(eval_results)

if __name__ == "__main__":
    evaluate_model()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 185)